<a href="https://colab.research.google.com/github/IANGECHUKI176/deeplearning/blob/main/pytorch/convnets/mobilenetv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creates a MobileNetV3 Model as defined in:
Andrew Howard, Mark Sandler, Grace Chu, Liang-Chieh Chen, Bo Chen, Mingxing Tan, Weijun Wang, Yukun Zhu, Ruoming Pang, Vijay Vasudevan, Quoc V. Le, Hartwig Adam. (2019).
Searching for MobileNetV3
> arXiv preprint https://arxiv.org/pdf/1905.02244.pdf

In [ ]:
import torch
import torch.nn as nn
import math
from torchsummary import summary

In [ ]:
def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

In [ ]:
class h_sigmoid(nn.Module):
    def __init__(self,inplace = True):
        super(h_sigmoid,self).__init__()
        self.relu = nn.ReLU6(inplace = inplace)

    def forward(self,x):
        return self.relu(x + 3)  / 6

In [ ]:
class h_swish(nn.Module):
    def __init__(self,inplace = True):
        super(h_swish,self).__init__()
        self.sigmoid = h_sigmoid(inplace = inplace)

    def forward(self,x):
        return x * self.sigmoid(x)

In [ ]:
class SELayer(nn.Module):
    def __init__(self,in_channel,reduction = 4):
        super(SELayer,self).__init__()
        mid_channel = _make_divisible(in_channel // reduction, 8)
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Sequential(
            nn.Linear(in_channel,mid_channel),
            nn.ReLU(inplace = True),
            nn.Linear(mid_channel,in_channel),
            h_sigmoid()
        )
    def forward(self,x):
        b,c,_,_ = x.size()
        y = self.avg_pool(x).view(b,c)
        y = self.fc(y).view(b,c,1,1)
        return x * y

In [ ]:
def conv_3x3_bn(ch_in,ch_out,stride):
    return nn.Sequential(
        nn.Conv2d(ch_in,ch_out,3,stride ,1,bias = False),
        nn.BatchNorm2d(ch_out),
        h_swish()
    )

In [ ]:
def conv_1x1_bn(ch_in,ch_out):
    return nn.Sequential(
        nn.Conv2d(ch_in,ch_out,1,1 ,0,bias = False),
        nn.BatchNorm2d(ch_out),
        h_swish()
    )

In [ ]:
class InvertedResidual(nn.Module):
    def __init__(self,ch_in,hidden_dim,ch_out,kernel_size,stride,use_se,use_hs):
        super(InvertedResidual,self).__init__()

        assert stride in [1,2]

        self.identity = stride == 1 and ch_in == ch_out
        if ch_in == hidden_dim:
            self.conv = nn.Sequential(
                #dw gcov
                nn.Conv2d(hidden_dim,hidden_dim,kernel_size,stride,(kernel_size - 1)//2,groups = hidden_dim,bias = False),
                nn.BatchNorm2d(hidden_dim),
                #se
                SELayer(hidden_dim) if use_hs else nn.Identity(),
                h_swish() if use_hs else nn.ReLU(inplace  = True),
                #pwlinear
                nn.Conv2d(hidden_dim,ch_out,1,1,0,bias = False),
                nn.BatchNorm2d(ch_out)
            )
        else:
            self.conv = nn.Sequential(
                #pw
                nn.Conv2d(ch_in,hidden_dim,1,1,0,bias = False),
                nn.BatchNorm2d(hidden_dim),
                h_swish() if use_hs else nn.ReLU(inplace = True),
                #dw gcov
                nn.Conv2d(hidden_dim,hidden_dim,kernel_size,stride,(kernel_size - 1)//2,groups = hidden_dim,bias = False),
                nn.BatchNorm2d(hidden_dim),
                #se
                SELayer(hidden_dim) if use_hs else nn.Identity(),
                h_swish() if use_hs else nn.ReLU(inplace  = True),
                #pwlinear
                nn.Conv2d(hidden_dim,ch_out,1,1,0,bias = False),
                nn.BatchNorm2d(ch_out)
            )
    def forward(self,x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)

In [ ]:
class MobileNetV3(nn.Module):
    def __init__(self,cfgs,mode ,num_classes = 10,width_mult = 1):
        super(MobileNetV3,self).__init__()
        self.cfgs = cfgs
        assert mode in ['large','small']
        input_channel = _make_divisible(16 * width_mult, 8)
        layers = [conv_3x3_bn(3, input_channel, 2)]

        block = InvertedResidual
        for k, t, c, use_se, use_hs, s in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            exp_size = _make_divisible(input_channel * t, 8)
            layers.append(block(input_channel,exp_size,output_channel,k,s,use_se,use_hs))
            input_channel = output_channel
        self.features = nn.Sequential(*layers)
        self.conv = conv_1x1_bn(input_channel,exp_size,)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        output_channel = {'large': 1280, 'small': 1024}
        output_channel = _make_divisible(output_channel[mode] * width_mult, 8) if width_mult > 1.0 else output_channel[mode]
        self.classifier = nn.Sequential(
            nn.Linear(exp_size,output_channel),
            h_swish(),
            nn.Dropout(0.2),
            nn.Linear(output_channel,num_classes)
        )
        self.initialize_weights()
    def forward(self,x):
        out = self.features(x)
        out = self.conv(out)
        out = self.avgpool(out)
        out = out.view(out.size(0),-1)
        out = self.classifier(out)
        return out
    def initialize_weights(self,):
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m,nn.BatchNorm2d):
                m.weight.data.fill_(0)
                m.bias.data.zero_()
            elif isinstance(m,nn.Linear):
                m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()

In [ ]:
def mobilenetv3_large(**kwargs):
    """
    Constructs a MobileNetV3-Large model
    """
    cfgs = [
        # k, t, c, SE, HS, s
        [3,   1,  16, 0, 0, 1],
        [3,   4,  24, 0, 0, 2],
        [3,   3,  24, 0, 0, 1],
        [5,   3,  40, 1, 0, 2],
        [5,   3,  40, 1, 0, 1],
        [5,   3,  40, 1, 0, 1],
        [3,   6,  80, 0, 1, 2],
        [3, 2.5,  80, 0, 1, 1],
        [3, 2.3,  80, 0, 1, 1],
        [3, 2.3,  80, 0, 1, 1],
        [3,   6, 112, 1, 1, 1],
        [3,   6, 112, 1, 1, 1],
        [5,   6, 160, 1, 1, 2],
        [5,   6, 160, 1, 1, 1],
        [5,   6, 160, 1, 1, 1]
    ]
    return MobileNetV3(cfgs, mode='large', **kwargs)

In [ ]:

def mobilenetv3_small(**kwargs):
    """
    Constructs a MobileNetV3-Small model
    """
    cfgs = [
        # k, t, c, SE, HS, s
        [3,    1,  16, 1, 0, 2],
        [3,  4.5,  24, 0, 0, 2],
        [3, 3.67,  24, 0, 0, 1],
        [5,    4,  40, 1, 1, 2],
        [5,    6,  40, 1, 1, 1],
        [5,    6,  40, 1, 1, 1],
        [5,    3,  48, 1, 1, 1],
        [5,    3,  48, 1, 1, 1],
        [5,    6,  96, 1, 1, 2],
        [5,    6,  96, 1, 1, 1],
        [5,    6,  96, 1, 1, 1],
    ]

    return MobileNetV3(cfgs, mode='small', **kwargs)

In [ ]:
# net = mobilenetv3_large()
# summary(net,(3,224,224))